### device_message_raw:
This table stores raw sensor readings from devices, including device identifiers, timestamps, message origin, sensor type, and recorded distance. It provides granular data on device activity and sensor measurements. These records are essential for building ML features, as they allow us to extract time-based sensor metrics, aggregate device behaviors, and correlate sensor readings with test events.

### rapid_step_tests_raw:
This table contains summaries of rapid step tests, with fields such as start and stop times, test duration, total steps, and an array of step intervals for each test, along with customer and device identifiers. It captures the performance and timing of step tests for each device and customer. This information is crucial for ML feature engineering, enabling us to link sensor activity to specific test windows and compute step timing statistics for predictive modeling.

In [0]:
%sql
CREATE SCHEMA IF NOT EXISTS workspace.silver;

In [0]:
%sql
CREATE TABLE IF NOT EXISTS workspace.silver.labeled_step_test (
  timestamp LONG COMMENT 'Exact time when the sensor reading was recorded (matches DeviceMessage.timestamp). Used to trace back to bronze.',
  deviceId STRING COMMENT 'The device that produced the sensor reading. Helps connect sensor data to a specific RapidStepTest.',
  bronze_record_id LONG COMMENT 'Optional surrogate key from the bronze DeviceMessage table. Useful for debugging and audit trails.',
  testId STRING COMMENT 'Optional ID generated during ETL to group all rows belonging to the same RapidStepTest session.',
  sensorType STRING COMMENT 'Type of sensor used (e.g., ultraSonicSensor). Comes from device_message_raw.',
  distance_cm INT COMMENT 'Numeric distance value (converted from string “1cm”). Represents the measured distance between device and foot.',
  step_label STRING COMMENT 'Label created by checking if timestamp is inside a RapidStepTest window. Values: "step" or "no_step".',
  source STRING COMMENT 'Indicates original table the row came from (e.g., "device"). Helps track data flow and detect issues.',
  startTime LONG COMMENT 'Start time of the Rapid Step Test window. Used to determine if a sensor reading happened during a step.',
  stopTime LONG COMMENT 'End time of the Rapid Step Test window. Paired with startTime for labeling.'
)
USING DELTA;


In [0]:
%sql
SELECT 
  column_name AS `Column Name`, 
  data_type AS `Data Type`,
  comment AS Comment
FROM 
  workspace.information_schema.columns
WHERE 
  table_schema = 'silver'
  AND table_name = 'labeled_step_test';
